In [16]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# Initialize Img2Vec with GPU
img2vec = Img2Vec()

# Read in an image (rgb format)
img = Image.open('CEDAR/3/forgeries_3_1.png').convert('RGB')
# Get a vector from img2vec, returned as a torch FloatTensor
vec = img2vec.get_vec(img, tensor=True)
# Or submit a list
# vectors = img2vec.get_vec(list_of_PIL_images)

c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
vec=vec.view(1,512)

In [20]:
vec=vec.squeeze()

In [22]:
vec.shape

torch.Size([512])

In [12]:
def img_to_vec(img):
    img2vec = Img2Vec()
    # Read in an image (rgb format)
    img = Image.open(img).convert('RGB')
    # Get a vector from img2vec, returned as a torch FloatTensor
    vec = img2vec.get_vec(img, tensor=True)
    vec=vec.view(vec.shape[0],vec.shape[1])
    return vec.squeeze()

In [27]:
import torch
def cosine(vec1,vec2):
    return torch.dot(vec1,vec2)/(torch.norm(vec1)*torch.norm(vec2)).item()

In [26]:
cosine(vec,vec).item()

1.0

In [24]:
import pandas as pd
df=pd.read_csv('CEDAR_test.csv')

In [15]:
df.head()

,Testing_Signature,Original_Signature,Ouput
0,CEDAR\10\original_10_11.png,CEDAR\10\original_10_22.png,1
1,CEDAR\28\forgeries_28_18.png,CEDAR\28\original_28_11.png,0
2,CEDAR\50\forgeries_50_12.png,CEDAR\50\original_50_16.png,0
3,CEDAR\37\forgeries_37_20.png,CEDAR\37\original_37_15.png,0
4,CEDAR\13\forgeries_13_4.png,CEDAR\13\original_13_17.png,0


In [31]:
import torch
import torch.nn as nn
from img2vec_pytorch import Img2Vec
from PIL import Image

class PretrainedModel(nn.Module):
    def __init__(self):
        super(PretrainedModel, self).__init__()
    
    def img_to_vec(self,img):
        img2vec = Img2Vec()
        # Read in an image (rgb format)
        img = Image.open(img).convert('RGB')
        # Get a vector from img2vec, returned as a torch FloatTensor
        vec = img2vec.get_vec(img, tensor=True)
        vec=vec.view(vec.shape[0],vec.shape[1])
        return vec.squeeze()
    
    def cosine(self,vec1,vec2):
        return torch.dot(vec1,vec2)/(torch.norm(vec1)*torch.norm(vec2)).item()

    def forward(self, x1, x2):
        # Forward pass for the two inputs separately
        output1 = self.img_to_vec(x1)
        output2 = self.img_to_vec(x2)
        return self.cosine(output1,output2)
        

In [38]:
from model import CustomDataModule
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Lambda(lambda img: img / 255.0)
    ])
dm = CustomDataModule('CEDAR_train.csv',transform=transform)
dm.setup()

# Get the training data loader
train_loader = dm.train_dataloader()

# Iterate over the batches and print the data
for batch_idx, (data1, data2, label) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print("Data 1:")
    print(data1.shape)
    print("Data 2:")
    print(data2.shape)
    print("Label:")
    print(label)
    break

Batch 1:
Data 1:
torch.Size([64, 3, 256, 256])
Data 2:
torch.Size([64, 3, 256, 256])
Label:
tensor([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1])


In [37]:
x=[]

y=[]
for batch_idx, (data1, data2, label) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    # print("Data 1:")
    # print(data1.shape)
    # print("Data 2:")
    # print(data2.shape)
    # print("Label:")
    # print(label)
    m=PretrainedModel()
    x.append(m(data1,data2))
    y.append(label)
    # break

Batch 1:


AttributeError: 'Tensor' object has no attribute 'read'

In [72]:
x=[]
y=[]
for i,j,k in zip(df['Testing_Signature'],df['Original_Signature'],df['Ouput']):
    vec1=img_to_vec(i)
    vec2=img_to_vec(j)
    x.append(cosine(vec1,vec2))
    y.append(k)
#save as the dataframe
df_new=pd.DataFrame({'cosine':x,'Ouput':y})

c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\Desktop\signature verification similarity\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
